# Errors

In this demo we are going to see how jetconf deals with errors in the data.

## Starting things up

Run:

    make build_container
    docker run \
        -p 8443:8443 \
        -e ONLINE_MODE=0 \
        rosetta -c /rosetta/tests/config.yaml

Now, let's export some environment variables we need:

In [1]:
export USER_CERT=../../tests/certs/test_user_curl.pem
export BASE_URL=https://ntc-rosetta-conf:8443

## Types of errors

There are three type of errors you can encounter:

1. Simple schema validation errors - For instance, a leaf being assigned a wrong type. This are performed on each operation.
2. Complex schema validation errors - Wrong identity value or value. This are performed on commit only.
3. Semantic errors - A duplicated key in a list, a missing leaf-ref, etc. This are performed on commit only.

### Simple schema validation errors

We are going to start trying to create a device with a number as name:

In [2]:
cat 3_errors/add_interface_eth0_bad_description.json

{
	"openconfig-interfaces:interface": {
		"name": "eth0",
		"config": {
			"name": "eth0",
			"description": 0,
			"type": "iana-if-type:ethernetCsmacd"
		}
	}
}


In [3]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @3_errors/add_interface_eth0_bad_description.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

{
    "ietf-restconf:errors": {
        "error": [
            {
                "error-type": "protocol",
                "error-tag": "invalid-value",
                "error-path": "/openconfig-interfaces:interfaces/interface/1/config/description",
                "error-message": "RawTypeError: expected string value"
            }
        ]
    }
}

In [4]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-reset

{
    "status": "OK"
}

### Complex schema validation errors

Now, let's try creating an interface with a very large MTU:

In [5]:
cat 3_errors/add_interface_eth0_large_mtu.json

{
	"openconfig-interfaces:interface": {
		"name": "eth0",
		"config": {
			"name": "eth0",
			"type": "iana-if-type:ethernetCsmacd",
			"mtu": 5465464564564645
		}
	}
}


In [6]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @3_errors/add_interface_eth0_large_mtu.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

In [7]:
# previous command succeeded, however, the commit will fail
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-commit

{
    "ietf-restconf:errors": {
        "error": [
            {
                "error-type": "protocol",
                "error-tag": "operation-failed",
                "error-app-tag": "invalid-type",
                "error-path": "/openconfig-interfaces:interfaces/interface/0/config/mtu",
                "error-message": "YangTypeError: expected uint16"
            }
        ]
    }
}

In [8]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-reset

{
    "status": "OK"
}

# Semantic errors

Now we are going to try to create a two devices with the same name. On commit it will complain the key is not unique (it doesn't matter if one of the element was previously commited or not):

In [9]:
cat 3_errors/add_interface_eth0.json

{
	"openconfig-interfaces:interface": {
		"name": "eth0",
		"config": {
			"name": "eth0",
			"type": "iana-if-type:ethernetCsmacd"
		}
	}
}


In [10]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @3_errors/add_interface_eth0.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

In [11]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @3_errors/add_interface_eth0.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

In [12]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

{
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "name": "eth0",
                "config": {
                    "name": "eth0",
                    "type": "iana-if-type:ethernetCsmacd"
                }
            },
            {
                "name": "eth0",
                "config": {
                    "name": "eth0",
                    "type": "iana-if-type:ethernetCsmacd"
                }
            }
        ]
    }
}

In [13]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-commit

{
    "ietf-restconf:errors": {
        "error": [
            {
                "error-type": "protocol",
                "error-tag": "invalid-value",
                "error-app-tag": "non-unique-key",
                "error-path": "/openconfig-interfaces:interfaces/interface",
                "error-message": "SemanticError: 'eth0'"
            }
        ]
    }
}

In [14]:
# ignore me, this discards the changes so the notebook can be rerun
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-reset

{
    "status": "OK"
}